<a href="https://colab.research.google.com/github/AmandaWu1123/early_warning_system/blob/main/Another_copy_of_Wizearly_bert_for_Interns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers torch pandas scikit-learn huggingface-hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 363.4/363.4 MB 84.5 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
https://github.com/AmandaWu1123/early_warning_system

In [ ]:
import pandas as pd
from transformers import BertTokenizer

# Load sample dataset
from google.colab import drive
drive.mount('/content/drive')
data = "/content/drive/My Drive/Wizearly Intern/dataset-bert(1).xlsx"
df = pd.read_excel(data)

# df = pd.DataFrame(data)

print(df.head())

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize sentences
max_length = 256
def tokenize_function(sentences):
    return tokenizer(sentences, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

tokenized_sentences = df['Sentence'].apply(lambda x: tokenize_function(x))
print(tokenized_sentences)

label_map = {
    "Security Emergencies": 0,
    "Weather Emergency": 1,
    "Pandemic-related factors": 2,
    "Local Events": 3,
    "Student Surveys": 4,
    "Teacher/Parent Feedback": 5,
    "School Funding Levels": 6,
    "Regional Absenteeism Trends": 7,
    "Social Media Activity": 8,
    "Student-Teacher Relationships": 9,
    "Peer relationships": 10,
    "Mental Health": 11,
    "Physical Health": 12,
    "School Climate": 13,
    "Teacher Quality": 14,
    "School Resources": 15,
    "Transportation Access": 16,
    "Home Stability": 17,
    "Family health issues": 18,
    "Parental Involvement": 19,
    "Behavioral Flags from Teachers": 20,
    "Participation in School Activities": 21,
    "Disciplinary Actions": 22,
    "Language Spoken at Home": 23,
    "Completion of Assignments and Homework": 24,
    "Special Education Status": 25,
    "Ethnicity": 26,
    "Socioeconomic status (SES)": 27
}

# Apply this mapping to the dataset
df['Label'] = df['Category'].map(label_map)



In [ ]:
import torch
from torch.utils.data import Dataset

class RiskFactorDataset(Dataset):
    def __init__(self, tokenized_sentences, labels):
        self.tokenized_sentences = tokenized_sentences
        self.labels = labels

    def __len__(self):
        return len(self.tokenized_sentences)

    def __getitem__(self, idx):
        return {
            'input_ids': self.tokenized_sentences[idx]['input_ids'].flatten(),
            'attention_mask': self.tokenized_sentences[idx]['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [ ]:
from transformers import BertForSequenceClassification
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
#from map_data import RiskFactorDataset
#from load_n_process import tokenized_sentences, df, tokenizer

# Load pretrained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# Create DataLoader
dataset = RiskFactorDataset(tokenized_sentences.tolist(), df['Label'].tolist())
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
model.train()
epochs = 10 #i changed it to 10
for epoch in range(epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

#Step 7: Classify New Sentences
model.eval()  # Set the model to evaluation mode

# Tokenize a new sentence
new_sentence = "My family has trouble making ends meet."
inputs = tokenizer(new_sentence, return_tensors="pt", truncation=True, padding=True, max_length=256)

# Get prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Predicted category
predicted_label = torch.argmax(logits, dim=1).item()

# Map back to category
category_map = category_map = {
    0: "Security Emergencies",
    1: "Weather Emergency",
    2: "Pandemic-related factors",
    3: "Local Events",
    4: "Student Surveys",
    5: "Teacher/Parent Feedback",
    6: "School Funding Levels",
    7: "Regional Absenteeism Trends",
    8: "Social Media Activity",
    9: "Student-Teacher Relationships",
    10: "Peer relationships",
    11: "Mental Health",
    12: "Physical Health",
    13: "School Climate",
    14: "Teacher Quality",
    15: "School Resources",
    16: "Transportation Access",
    17: "Home Stability",
    18: "Family health issues",
    19: "Parental Involvement",
    20: "Behavioral Flags from Teachers",
    21: "Participation in School Activities",
    22: "Disciplinary Actions",
    23: "Language Spoken at Home",
    24: "Completion of Assignments and Homework",
    25: "Special Education Status",
    26: "Ethnicity",
    27: "Socioeconomic status (SES)"
}

predicted_category = category_map[predicted_label]

print(f"Predicted category: {predicted_category}")


# Evaluate Models
# from sklearn.metrics import accuracy_score, classification_report

# # Assume y_true and y_pred are your true labels and predicted labels
# y_true = [0, 1, 2, 3]  # Replace with actual test labels
# y_pred = [0, 1, 2, 3]  # Replace with actual model predictions

# # Accuracy
# print(f"Accuracy: {accuracy_score(y_true, y_pred)}")

# # Classification report
# print(classification_report(y_true, y_pred, target_names=category_map.values()))


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 3.3690309524536133
Epoch: 0, Loss: 3.3727407455444336
Epoch: 0, Loss: 3.223834753036499
Epoch: 0, Loss: 3.149230718612671
Epoch: 0, Loss: 2.784121513366699
Epoch: 0, Loss: 3.4125399589538574
Epoch: 0, Loss: 3.4151506423950195
Epoch: 0, Loss: 3.1871793270111084
Epoch: 0, Loss: 3.590083599090576
Epoch: 0, Loss: 3.3247456550598145
Epoch: 0, Loss: 3.560236692428589
Epoch: 0, Loss: 3.396070957183838
Epoch: 0, Loss: 2.6592345237731934
Epoch: 0, Loss: 3.1320130825042725
Epoch: 0, Loss: 3.8655412197113037
Epoch: 0, Loss: 3.1806764602661133
Epoch: 0, Loss: 3.706322431564331
Epoch: 0, Loss: 3.107046604156494
Epoch: 0, Loss: 3.524509906768799
Epoch: 0, Loss: 3.3772926330566406
Epoch: 0, Loss: 3.4438915252685547
Epoch: 0, Loss: 2.755633592605591
Epoch: 0, Loss: 3.590250015258789
Epoch: 0, Loss: 3.3874824047088623
Epoch: 0, Loss: 2.98738169670105
Epoch: 0, Loss: 3.0001001358032227
Epoch: 0, Loss: 3.0543923377990723
Epoch: 0, Loss: 3.3050522804260254
Epoch: 0, Loss: 3.492490768432617

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Save the model's state dictionary to a file
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/bert/bert_risk_factor_model.pth')


# **Loading the saved parameter**

In [ ]:
from transformers import BertForSequenceClassification
import torch
from transformers import BertTokenizer
import pandas as pd

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

label_map = {
    "Security Emergencies": 0,
    "Weather Emergency": 1,
    "Pandemic-related factors": 2,
    "Local Events": 3,
    "Student Surveys": 4,
    "Teacher/Parent Feedback": 5,
    "School Funding Levels": 6,
    "Regional Absenteeism Trends": 7,
    "Social Media Activity": 8,
    "Student-Teacher Relationships": 9,
    "Peer relationships": 10,
    "Mental Health": 11,
    "Physical Health": 12,
    "School Climate": 13,
    "Teacher Quality": 14,
    "School Resources": 15,
    "Transportation Access": 16,
    "Home Stability": 17,
    "Family health issues": 18,
    "Parental Involvement": 19,
    "Behavioral Flags from Teachers": 20,
    "Participation in School Activities": 21,
    "Disciplinary Actions": 22,
    "Language Spoken at Home": 23,
    "Completion of Assignments and Homework": 24,
    "Special Education Status": 25,
    "Ethnicity": 26,
    "Socioeconomic status (SES)": 27
}

from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/bert/bert_risk_factor_model.pth'))

model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-aa7464e2832b>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We reco

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# ipython-input-2-ca50d0db9661
from transformers import BertTokenizer

# Tokenize a new sentence
new_sentence = "Security incidents can create a feeling of distrust among students, leading to disengagement."

# Initialize the tokenizer here:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # This line was missing

inputs = tokenizer(new_sentence, return_tensors="pt", truncation=True, padding=True, max_length=256)

# Get prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Predicted category
predicted_label = torch.argmax(logits, dim=1).item()

category_map = category_map = {
    0: "Security Emergencies",
    1: "Weather Emergency",
    2: "Pandemic-related factors",
    3: "Local Events",
    4: "Student Surveys",
    5: "Teacher/Parent Feedback",
    6: "School Funding Levels",
    7: "Regional Absenteeism Trends",
    8: "Social Media Activity",
    9: "Student-Teacher Relationships",
    10: "Peer relationships",
    11: "Mental Health",
    12: "Physical Health",
    13: "School Climate",
    14: "Teacher Quality",
    15: "School Resources",
    16: "Transportation Access",
    17: "Home Stability",
    18: "Family health issues",
    19: "Parental Involvement",
    20: "Behavioral Flags from Teachers",
    21: "Participation in School Activities",
    22: "Disciplinary Actions",
    23: "Language Spoken at Home",
    24: "Completion of Assignments and Homework",
    25: "Special Education Status",
    26: "Ethnicity",
    27: "Socioeconomic status (SES)"
}
predicted_category = category_map[predicted_label]

print(f"Predicted category: {predicted_category}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

NameError: name 'torch' is not defined

In [ ]:
# Tokenize a new sentence
new_sentence = "Security incidents can create a feeling of distrust among students, leading to disengagement."
inputs = tokenizer(new_sentence, return_tensors="pt", truncation=True, padding=True, max_length=256)

# Get prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Predicted category
predicted_label = torch.argmax(logits, dim=1).item()

category_map = category_map = {
    0: "Security Emergencies",
    1: "Weather Emergency",
    2: "Pandemic-related factors",
    3: "Local Events",
    4: "Student Surveys",
    5: "Teacher/Parent Feedback",
    6: "School Funding Levels",
    7: "Regional Absenteeism Trends",
    8: "Social Media Activity",
    9: "Student-Teacher Relationships",
    10: "Peer relationships",
    11: "Mental Health",
    12: "Physical Health",
    13: "School Climate",
    14: "Teacher Quality",
    15: "School Resources",
    16: "Transportation Access",
    17: "Home Stability",
    18: "Family health issues",
    19: "Parental Involvement",
    20: "Behavioral Flags from Teachers",
    21: "Participation in School Activities",
    22: "Disciplinary Actions",
    23: "Language Spoken at Home",
    24: "Completion of Assignments and Homework",
    25: "Special Education Status",
    26: "Ethnicity",
    27: "Socioeconomic status (SES)"
}
predicted_category = category_map[predicted_label]

print(f"Predicted category: {predicted_category}")

NameError: name 'tokenizer' is not defined